In [1]:
import json
import numpy as np

In [2]:
friend_data = np.genfromtxt('grupee_data/friends.csv', delimiter=',', dtype=int)
friendships = [[] for _ in range(np.max(friend_data[:,0])+1)]

for id in set(friend_data[:,0]):
    friendships[id] = [id, friend_data[friend_data[:, 0] == id, 1]]

with open('grupee_data/preferences.json') as file:
    preferences = json.load(file)

with open('grupee_data/n_concerts.txt') as file:
    n_concerts = file.read().split('\n')[1:-1]
    concerts = np.array([(x.split(':')[0], x.split(':')[1]) for x in n_concerts])
    file.close()

In [3]:
friendship_matrix = np.empty((len(friendships), len(friendships)), dtype=int)

for i, id in enumerate(friendships):
    row = np.zeros(len(friendships))
    if id:
        row[id[1]] = 1
        friendship_matrix[i] = row
    else:
        friendship_matrix[i] = row
        
friendship_matrix = friendship_matrix + friendship_matrix.T - np.diag(friendship_matrix.diagonal())

<h3>Define likelihoods</h3>

In [1]:
visit_concerts = {
    'none': .2,
    'one': 1.8,
    'both': 39.3,
}

***
<h2>Start with Processing</h2>

<h3>Laplacian Matrix</h3>

In [ ]:
degreeMat = np.diag(np.sum(friendship_matrix, axis=1))
laplacian = degreeMat - friendship_matrix

degrees = np.empty((len(friendships), 2))

for id in range(len(friendships)):
    degrees[id] = (id, laplacian[id][id])

print(f'12% of {len(friendships)}: {len(friendships) * .12}')
sortedDegrees = sorted(degrees, key=lambda tup: tup[1])
mostInfluential = np.array(sortedDegrees[len(friendships)-round(len(friendships) * .12):], dtype=int)
#np.savetxt('a_team_5.txt', np.array(list(reversed(mostInfluential[:,0]))), delimiter='\n', fmt='%i')

<h3>Some other model...<h3>